In [1]:
import pandas as pd 
import numpy as np 
import os
import json
from tqdm import tqdm

In [2]:
tqdm.pandas(desc="Progress")

In [3]:
# For the file directory assumptions to work correctly, you will need the contents of parler_posts_json.zip extracted to the directory parler_posts_json located in the same directory as this file.
cwd = os.getcwd()
posts_dir = cwd + '/data/parler_posts_json/'
meta_dir = cwd + '/data/metadata/'

In [11]:
len(os.listdir(meta_dir))

1032524

In [38]:
#testfile = posts_dir + os.listdir(posts_dir)[1]
#test = pd.read_json(testfile)

frames = ['GPSLatitude',
    'GPSLongitude',
    'GPSAltitude',
    'GPSDateStamp',
    'GPSLatitudeRef',
    'GPSLongitudeRef',
    'GPSAltitudeRef',
    'GPSTimeStamp',
    'GPSTrack',
    'GPSSpeed',
    'GPSImgDirection',
    'GPSPitch',
    'GPSTrackRef',
    'GPSSpeedRef',
    'GPSImgDirectionRef',
    'GPSRoll',
    'AmbientTemperature',
    'CameraElevationAngle',]

def data_iterator(path):
    dumbs = []
    for record in tqdm(os.listdir(path)):
        if record.endswith('.json'):
            output = {'file': record}
            try:
                with open(meta_dir + record) as f:
                    data = json.load(f)[0]
                for frame in frames:
                    if frame in data:
                        output[frame] = data[frame]
                if len(output.keys()) > 1:
                    for date in ['CreateDate', 'ModifyDate']:
                        if date in data:
                            if not data[date] == '0000:00:00 00:00:00':
                                output[date] = data[date]
                    yield output
            except:
                print(f'** File {record} could not be handled for some dumb reason')
                dumbs.append(record)
metadata = pd.DataFrame(data_iterator(meta_dir))
metadata.head()

100%|██████████| 1032524/1032524 [01:01<00:00, 16775.34it/s]


,file,GPSLatitude,GPSLongitude,GPSAltitude,GPSAltitudeRef,CreateDate,ModifyDate,GPSDateStamp,GPSLatitudeRef,GPSLongitudeRef,GPSTimeStamp,GPSImgDirection,GPSImgDirectionRef,GPSSpeed,GPSSpeedRef
0,meta-27PknKIOwHt6.json,"34 deg 4' 20.64"" N","118 deg 24' 9.36"" W",78 m,Above Sea Level,2020:10:31 23:02:29,2020:10:31 23:03:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,meta-m3Wq53jjPnpw.json,"34 deg 2' 50.28"" N","118 deg 15' 35.64"" W",77.904 m,Above Sea Level,2020:12:21 23:22:36,2020:12:21 23:22:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,meta-kpKT3stt5LXq.json,"26 deg 20' 2.40"" N","80 deg 17' 41.64"" W",3.848 m,Above Sea Level,2020:12:04 20:49:20,2020:12:04 20:49:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,meta-qgU5924Q8jQh.json,"33 deg 58' 57.72"" N","118 deg 12' 33.84"" W",47.735 m,Above Sea Level,2020:11:04 07:29:46,2020:11:04 07:29:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,meta-XfeRIAoVLtei.json,"22 deg 21' 16.56"" S","42 deg 20' 8.52"" W",641.108 m,Above Sea Level,2020:07:02 12:50:02,2020:07:02 12:50:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
metadata

,file,GPSLatitude,GPSLongitude,GPSAltitude,GPSAltitudeRef,CreateDate,ModifyDate,GPSDateStamp,GPSLatitudeRef,GPSLongitudeRef,GPSTimeStamp,GPSImgDirection,GPSImgDirectionRef,GPSSpeed,GPSSpeedRef
0,meta-27PknKIOwHt6.json,"34 deg 4' 20.64"" N","118 deg 24' 9.36"" W",78 m,Above Sea Level,2020:10:31 23:02:29,2020:10:31 23:03:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,meta-m3Wq53jjPnpw.json,"34 deg 2' 50.28"" N","118 deg 15' 35.64"" W",77.904 m,Above Sea Level,2020:12:21 23:22:36,2020:12:21 23:22:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,meta-kpKT3stt5LXq.json,"26 deg 20' 2.40"" N","80 deg 17' 41.64"" W",3.848 m,Above Sea Level,2020:12:04 20:49:20,2020:12:04 20:49:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,meta-qgU5924Q8jQh.json,"33 deg 58' 57.72"" N","118 deg 12' 33.84"" W",47.735 m,Above Sea Level,2020:11:04 07:29:46,2020:11:04 07:29:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,meta-XfeRIAoVLtei.json,"22 deg 21' 16.56"" S","42 deg 20' 8.52"" W",641.108 m,Above Sea Level,2020:07:02 12:50:02,2020:07:02 12:50:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68463,meta-JMfLmefCzvMW.json,"35 deg 12' 59.40"" S","138 deg 32' 18.96"" E",54.644 m,Above Sea Level,2020:06:30 07:03:29,2020:06:30 07:04:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68464,meta-33oluYHSIrDn.json,"33 deg 48' 50.04"" N","117 deg 32' 43.08"" W",389.691 m,Above Sea Level,2020:07:17 09:44:45,2020:07:17 09:45:19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68465,meta-pMdvwJuktYPj.json,"38 deg 53' 26.52"" N","77 deg 0' 20.88"" W",28.555 m,Above Sea Level,2020:12:12 19:53:48,2020:12:12 19:55:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68466,meta-50A0Fl2Fcg89.json,"49 deg 10' 26.76"" N","122 deg 40' 47.64"" W",14.619 m,Above Sea Level,2020:11:25 21:05:29,2020:11:25 21:06:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
